In [1]:
import os
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class YoloDataset(Dataset):
    def init(self, imagesdir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.images = os.listdir(images_dir)
        self.transform = transform

    def len(self):
        return len(self.images)

    def getitem(self, idx):
        img_file = self.images[idx]
        img_path = os.path.join(self.images_dir, img_file)
        label_file = img_file.replace('.jpg', '.txt')
        label_path = os.path.join(self.labels_dir, label_file)

#Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w,  = image.shape

#Load labels
        boxes = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                boxes.append([x_center, y_center, width, height, int(class_id)])
        boxes = np.array(boxes)

#Apply transformations
        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes[:, :4], class_labels=boxes[:, 4])
            image = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['class_labels']

#Convert to tensor
            targets = []
            for bbox, label in zip(boxes, labels):
                targets.append([label, *bbox])
            targets = torch.tensor(targets)

            return image, targets
        else:
            return image, boxes

ModuleNotFoundError: No module named 'cv2'